In [5]:
from anytree.node.nodemixin import NodeMixin

In [1]:
# from ltp import LTP
# ltp = LTP()

In [1]:
import os, torch
from ltp import LTP
from ltp.ltp import no_gard
from ltp.utils.seqeval import get_entities
class LtpExt(LTP):
    @no_gard
    def ner_bio(self, hidden: dict):
        # 命名实体识别
        word_length = torch.as_tensor(hidden['word_length'], device=self.device)
        ner_output = self.model.ner_decoder(hidden['word_input'], word_length)
        ner_output = torch.argmax(ner_output, dim=-1).cpu().numpy()
        ner_output = self._convert_idx_to_name(ner_output, hidden['word_length'], self.ner_vocab)
        # return [get_entities(ner) for ner in ner_output]
        return ner_output

ltp=LtpExt()

In [32]:
sents="他叫汤姆去拿外衣。"
seg, hidden = ltp.seg([sents])
print(seg)
# hidden
ltp.ner_bio(hidden)

[['他', '叫', '汤姆', '去', '拿', '外衣', '。']]


[['O', 'O', 'S-Nh', 'O', 'O', 'O', 'O']]

In [33]:
ltp.ner(hidden)

[[('Nh', 2, 2)]]

In [2]:
# import ltp as ltpns
sents = ltp.sent_split(["他叫汤姆去拿外衣。", "汤姆生病了。他去了医院。"])
sents

['他叫汤姆去拿外衣。', '汤姆生病了。', '他去了医院。']

In [8]:
from typing import Text, Any, Dict, List, Union, Optional, Tuple, Set
from anytree.node.nodemixin import NodeMixin
from anytree.node.util import _repr
class AnalBaseNode(NodeMixin, object):
    name: str

    tags: Set[Text]={}
    term: str='_'
    description: str=''

    def __init__(self, name, parent=None, children=None, **kwargs):
        self.name=name
        self.__dict__.update(kwargs)        
        self.parent = parent
        if children:
            self.children = children

    def __repr__(self):
        return _repr(self)

class AnalKgNode(AnalBaseNode):
    def __repr__(self):
        return _repr(self)

In [3]:
sents="他叫汤姆去拿外衣。"
seg, hidden = ltp.seg([sents])
print(seg)
hidden

[['他', '叫', '汤姆', '去', '拿', '外衣', '。']]


{'word_cls': tensor([[[-3.8648e-01, -3.1556e-01,  3.4055e-01,  5.5514e-01, -3.5819e-01,
            2.4156e-01,  3.7879e+00, -2.7905e-01, -2.9810e-02,  3.8253e-01,
            3.1881e-01, -9.3869e-02,  2.3798e-01, -3.1773e-01,  4.0632e-01,
            2.6014e-01,  1.3534e-01, -1.7425e-01,  4.1107e-01,  1.9668e-01,
           -4.1747e-01, -4.8434e-02, -1.1819e-01, -4.2857e-01,  1.5762e-01,
           -1.6501e-01,  6.4266e-01, -1.6018e-01, -8.4843e-02,  5.5382e-02,
           -2.3941e-01,  1.7425e-04, -9.9947e-03,  3.1917e-01, -1.9612e-01,
           -8.2507e-02, -2.3384e-01, -2.2817e-01,  4.3930e-02,  1.7800e-01,
            3.8812e-01,  3.2865e-01, -1.0002e-01, -2.0737e-01,  2.0631e-01,
            4.0198e-01, -2.3285e-01,  2.0695e-01,  6.8277e-02,  1.6078e-01,
            1.6904e-02, -1.5677e-01,  2.4631e-01,  2.1048e-01,  6.3682e-02,
            8.8806e-02, -4.1405e-01, -2.1406e-01,  2.7013e-01,  8.4621e-02,
            4.3258e-02, -2.8607e-01,  8.0837e-01,  3.8348e-01, -2.1228e-01,


In [4]:
ltp.sdp(hidden, graph=False)

[[(1, 2, 'Agt'),
  (2, 0, 'Root'),
  (3, 2, 'Datv'),
  (4, 2, 'eEfft'),
  (5, 4, 'eEfft'),
  (6, 5, 'Pat'),
  (7, 2, 'mPunc')]]

In [5]:
ltp.pos(hidden)

[['r', 'v', 'nh', 'v', 'v', 'n', 'wp']]

In [2]:
def proc_sents(sents):
    seg, hidden = ltp.seg([sents])
    print(seg[0])
    sdp = ltp.sdp(hidden, graph=False)
    for start, end, tag in sdp[0]:
        print(seg[0][start-1],f"({tag})", seg[0][end-1])

proc_sents("他叫汤姆去拿外衣。")

['他', '叫', '汤姆', '去', '拿', '外衣', '。']
他 (Agt) 叫
叫 (Root) 。
汤姆 (Datv) 叫
去 (eEfft) 叫
拿 (eEfft) 去
外衣 (Pat) 拿
。 (mPunc) 叫


In [10]:
# sent_ls=.split('。')
text='''2018年4月22日，2017年全球最赚钱企业排行榜第15。
2018年5月29日，《2018年BrandZ全球最具价值品牌100强》第4位。
2018年7月19日，《财富》世界500强排行榜位列71位。2018年12月18日，《2018世界品牌500强》第4位。'''
sent_ls = ltp.sent_split([text])
print(len(sent_ls), sent_ls)
segs, hidden = ltp.seg([s for s in sent_ls if s!=''])
sdps = ltp.sdp(hidden, graph=False)
for seg,sdp in zip(segs, sdps):
    print(seg)
    for start, end, tag in sdp:
        print('\t', seg[start-1],f"({tag})", seg[end-1])

4 ['2018年4月22日，2017年全球最赚钱企业排行榜第15。', '2018年5月29日，《2018年BrandZ全球最具价值品牌100强》第4位。', '2018年7月19日，《财富》世界500强排行榜位列71位。', '2018年12月18日，《2018世界品牌500强》第4位。']
['2018年', '4月', '22日', '，', '2017年', '全球', '最', '赚钱', '企业', '排行榜', '第 15', '。', '', '', '', '', '']
	 2018年 (Mod) 22日
	 4月 (Mod) 22日
	 22日 (Time) 
	 ， (mPunc) 22日
	 2017年 (Time) 排行榜
	 全球 (Sco) 企业
	 最 (mDepd) 赚钱
	 赚钱 (Mod) 企业
	 企业 (Mod) 排行榜
	 排行榜 (Exp) 第 15
	 第 15 (Meas) 
	 。 (mPunc) 第 15
	  (mPunc) 第 15
	  (Meas) 第 15
	  (mDepd) 第 15
	  (Root) 
	  (mPunc) 
['2018年', '5月', '29日', '，', '《', '2018年', 'BrandZ', '全球', '最', '具', '价值', '品牌', '100', '强', '》', '第 4', '位', '。']
	 2018年 (Mod) 29日
	 5月 (Mod) 29日
	 29日 (Time) 位
	 ， (mPunc) 29日
	 《 (mPunc) 具
	 2018年 (Time) 具
	 BrandZ (Mod) 全球
	 全球 (Loc) 具
	 最 (mDepd) 具
	 具 (rExp) 强
	 价值 (Mod) 品牌
	 品牌 (Link) 具
	 100 (Meas) 强
	 强 (eSymm) 位
	 》 (mPunc) 强
	 第 4 (Meas) 位
	 位 (Root) 。
	 。 (mPunc) 位
['2018年', '7月', '19日', '，', '《', '财富', '》', '世界', '500', '强', '排行榜', '位', '列', '71', '位', '。', '', '', '', '']
	

In [11]:
proc_sents("我爱自然语言处理。元芳你怎么看")

['我', '爱', '自然', '语言', '处理', '。', '元芳', '你', '怎么', '看']
我 (Agt) 爱
爱 (Root) 看
自然 (Mod) 语言
语言 (Tool) 处理
处理 (Cont) 爱
。 (mPunc) 爱
元芳 (Agt) 看
你 (Agt) 看
怎么 (mDepd) 看
看 (eAdvt) 爱


In [12]:
term_exprs='''关系类型	Tag	Description	Example
施事关系	Agt	Agent	我送她一束花 (我 <-- 送)
当事关系	Exp	Experiencer	我跑得快 (跑 --> 我)
感事关系	Aft	Affection	我思念家乡 (思念 --> 我)
领事关系	Poss	Possessor	他有一本好读 (他 <-- 有)
受事关系	Pat	Patient	他打了小明 (打 --> 小明)
客事关系	Cont	Content	他听到鞭炮声 (听 --> 鞭炮声)
成事关系	Prod	Product	他写了本小说 (写 --> 小说)
源事关系	Orig	Origin	我军缴获敌人四辆坦克 (缴获 --> 坦克)
涉事关系	Datv	Dative	他告诉我个秘密 ( 告诉 --> 我 )
比较角色	Comp	Comitative	他成绩比我好 (他 --> 我)
属事角色	Belg	Belongings	老赵有俩女儿 (老赵 <-- 有)
类事角色	Clas	Classification	他是中学生 (是 --> 中学生)
依据角色	Accd	According	本庭依法宣判 (依法 <-- 宣判)
缘故角色	Reas	Reason	他在愁女儿婚事 (愁 --> 婚事)
意图角色	Int	Intention	为了金牌他拼命努力 (金牌 <-- 努力)
结局角色	Cons	Consequence	他跑了满头大汗 (跑 --> 满头大汗)
方式角色	Mann	Manner	球慢慢滚进空门 (慢慢 <-- 滚)
工具角色	Tool	Tool	她用砂锅熬粥 (砂锅 <-- 熬粥)
材料角色	Malt	Material	她用小米熬粥 (小米 <-- 熬粥)
材料角色	Matl	Material	她用小米熬粥 (小米 <-- 熬粥)
时间角色	Time	Time	唐朝有个李白 (唐朝 <-- 有)
空间角色	Loc	Location	这房子朝南 (朝 --> 南)
历程角色	Proc	Process	火车正在过长江大桥 (过 --> 大桥)
趋向角色	Dir	Direction	部队奔向南方 (奔 --> 南)
范围角色	Sco	Scope	产品应该比质量 (比 --> 质量)
数量角色	Quan	Quantity	一年有365天 (有 --> 天)
数量数组	Qp	Quantity-phrase	三本书 (三 --> 本)
频率角色	Freq	Frequency	他每天看书 (每天 <-- 看)
顺序角色	Seq	Sequence	他跑第一 (跑 --> 第一)
描写角色	Desc(Feat)	Description	他长得胖 (长 --> 胖)
宿主角色	Host	Host	住房面积 (住房 <-- 面积)
名字修饰角色	Nmod	Name-modifier	果戈里大街 (果戈里 <-- 大街)
时间修饰角色	Tmod	Time-modifier	星期一上午 (星期一 <-- 上午)
反角色	r + main role		打篮球的小姑娘 (打篮球 <-- 姑娘)
嵌套角色	d + main role		爷爷看见孙子在跑 (看见 --> 跑)
并列关系	eCoo	event Coordination	我喜欢唱歌和跳舞 (唱歌 --> 跳舞)
选择关系	eSelt	event Selection	您是喝茶还是喝咖啡 (茶 --> 咖啡)
等同关系	eEqu	event Equivalent	他们三个人一起走 (他们 --> 三个人)
先行关系	ePrec	event Precedent	首先，先
顺承关系	eSucc	event Successor	随后，然后
递进关系	eProg	event Progression	况且，并且
转折关系	eAdvt	event adversative	却，然而
原因关系	eCau	event Cause	因为，既然
结果关系	eResu	event Result	因此，以致
推论关系	eInf	event Inference	才，则
条件关系	eCond	event Condition	只要，除非
假设关系	eSupp	event Supposition	如果，要是
让步关系	eConc	event Concession	纵使，哪怕
手段关系	eMetd	event Method	
目的关系	ePurp	event Purpose	为了，以便
割舍关系	eAban	event Abandonment	与其，也不
选取关系	ePref	event Preference	不如，宁愿
总括关系	eSum	event Summary	总而言之
分叙关系	eRect	event Recount	例如，比方说
连词标记	mConj	Recount Marker	和，或
的字标记	mAux	Auxiliary	的，地，得
介词标记	mPrep	Preposition	把，被
语气标记	mTone	Tone	吗，呢
时间标记	mTime	Time	才，曾经
范围标记	mRang	Range	都，到处
程度标记	mDegr	Degree	很，稍微
频率标记	mFreq	Frequency Marker	再，常常
趋向标记	mDir	Direction Marker	上去，下来
插入语标记	mPars	Parenthesis Marker	总的来说，众所周知
否定标记	mNeg	Negation Marker	不，没，未
情态标记	mMod	Modal Marker	幸亏，会，能
标点标记	mPunc	Punctuation Marker	，。！
重复标记	mPept	Repetition Marker	走啊走 (走 --> 走)
多数标记	mMaj	Majority Marker	们，等
实词虚化标记	mVain	Vain Marker	
离合标记	mSepa	Seperation Marker	吃了个饭 (吃 --> 饭) 洗了个澡 (洗 --> 澡)
根节点	Root	Root	全句核心节点'''.split('\n')
rows=[r.split('\t') for r in term_exprs]
terms_map={t[1]:f"{t[2]}_{t[0]}" for t in rows}
terms_map

{'Tag': 'Description_关系类型',
 'Agt': 'Agent_施事关系',
 'Exp': 'Experiencer_当事关系',
 'Aft': 'Affection_感事关系',
 'Poss': 'Possessor_领事关系',
 'Pat': 'Patient_受事关系',
 'Cont': 'Content_客事关系',
 'Prod': 'Product_成事关系',
 'Orig': 'Origin_源事关系',
 'Datv': 'Dative_涉事关系',
 'Comp': 'Comitative_比较角色',
 'Belg': 'Belongings_属事角色',
 'Clas': 'Classification_类事角色',
 'Accd': 'According_依据角色',
 'Reas': 'Reason_缘故角色',
 'Int': 'Intention_意图角色',
 'Cons': 'Consequence_结局角色',
 'Mann': 'Manner_方式角色',
 'Tool': 'Tool_工具角色',
 'Malt': 'Material_材料角色',
 'Matl': 'Material_材料角色',
 'Time': 'Time_时间角色',
 'Loc': 'Location_空间角色',
 'Proc': 'Process_历程角色',
 'Dir': 'Direction_趋向角色',
 'Sco': 'Scope_范围角色',
 'Quan': 'Quantity_数量角色',
 'Qp': 'Quantity-phrase_数量数组',
 'Freq': 'Frequency_频率角色',
 'Seq': 'Sequence_顺序角色',
 'Desc(Feat)': 'Description_描写角色',
 'Host': 'Host_宿主角色',
 'Nmod': 'Name-modifier_名字修饰角色',
 'Tmod': 'Time-modifier_时间修饰角色',
 'r + main role': '_反角色',
 'd + main role': '_嵌套角色',
 'eCoo': 'event Coordination_并列关系',
 'eSelt': 'eve

In [13]:
from anytree import Node, RenderTree
nodecls=AnalKgNode
def build_tree(sents, verbose=True, lang='zh'):
    if not sents.endswith(('.', '?', '。', '？', '！')):
        sents=sents+'.'
    seg, hidden = ltp.seg([sents])
    words=seg[0]
    nodes = [nodecls(word, lang=lang) for word in words]
    sdp = ltp.sdp(hidden, graph=False)
    root=None
    for start, end, tag in sdp[0]:
        if verbose:
            print(seg[0][start-1],f"({tag})", seg[0][end-1])
        if tag=='mPunc':
            continue
        nodes[start-1].parent=nodes[end-1]
        nodes[start-1].term=tag
        if tag in terms_map:
            nodes[start-1].description=terms_map[tag]
        else:
            nodes[start-1].description='_'
        if tag=='Root':
            root=nodes[end-1]
        
    return root

def print_sdp(sents, detail=True, verbose=False, lang='zh'):
    root=build_tree(sents, verbose=verbose)
    for pre, _, node in RenderTree(root):
        print("%s%s(%s): %s" % (pre, node.term, 
                                node.description if detail else '_', 
                                node.name))
# print(RenderTree(self, style=AsciiStyle()).by_attr(
#             lambda n: f"{n.dependency_relation}: {n.text} "
#                       f"({additional(n)}, {n.upos.lower()})"))

print_sdp('他叫汤姆去拿外衣。')

_(): 。
└── Root(Root_根节点): 叫
    ├── Agt(Agent_施事关系): 他
    ├── Datv(Dative_涉事关系): 汤姆
    └── eEfft(_): 去
        └── eEfft(_): 拿
            └── Pat(Patient_受事关系): 外衣


In [14]:
print_sdp('唐朝有个李白', True)

_(): .
└── Root(Root_根节点): 有
    ├── Time(Time_时间角色): 唐朝
    └── Exp(Experiencer_当事关系): 李白
        └── Meas(_): 个


In [15]:
print_sdp('我喜欢唱歌和跳舞', True, True)

我 (Agt) 喜欢
喜欢 (Root) .
唱歌 (dCont) 喜欢
和 (mRela) 跳舞
跳舞 (eSymm) 唱歌
. (mPunc) 喜欢
_(): .
└── Root(Root_根节点): 喜欢
    ├── Agt(Agent_施事关系): 我
    └── dCont(_): 唱歌
        └── eSymm(_): 跳舞
            └── mRela(_): 和


In [16]:
print_sdp('您是喝茶还是喝咖啡 ')

_(): .
└── Root(Root_根节点): 喝茶
    ├── Agt(Agent_施事关系): 您
    ├── mRela(_): 是
    └── eSymm(_): 喝
        ├── mRela(_): 还是
        └── Pat(Patient_受事关系): 咖啡


In [17]:
print_sdp('他们三个人一起走 ')

_(): .
└── Root(Root_根节点): 走
    ├── Agt(Agent_施事关系): 人
    │   ├── Exp(Experiencer_当事关系): 他们
    │   └── Meas(_): 个
    │       └── Meas(_): 三
    └── Mann(Manner_方式角色): 一起


In [18]:
print_sdp('我家的住房面积是80平米')

_(): .
└── Root(Root_根节点): 是
    ├── Exp(Experiencer_当事关系): 面积
    │   └── Mod(_): 住房
    │       └── Exp(Experiencer_当事关系): 我家
    │           └── mDepd(_): 的
    └── Meas(_): 平米
        └── Meas(_): 80


In [19]:
sents=['他长得胖', '他跑第一', '他每天看书 ', '为了金牌他拼命努力']
for s in sents:
    print_sdp(s)

_(): .
└── Root(Root_根节点): 长
    ├── Exp(Experiencer_当事关系): 他
    ├── mDepd(_): 得
    └── Mod(_): 胖
_(): .
└── Root(Root_根节点): 跑
    ├── Agt(Agent_施事关系): 他
    └── Meas(_): 第一
_(): .
└── Root(Root_根节点): 看
    ├── Agt(Agent_施事关系): 他
    ├── Meas(_): 每天
    └── Cont(Content_客事关系): 书
_(): .
└── Root(Root_根节点): 努力
    ├── Datv(Dative_涉事关系): 金牌
    │   └── mRela(_): 为了
    ├── Agt(Agent_施事关系): 他
    └── mNeg(Negation Marker_否定标记): 拼命


In [20]:
sents=['我吃了个巧克力味冰淇淋。', '今天我去电影院看了浓情巧克力。', 
       '她用小米熬粥', '她用美的砂锅熬粥', '我军缴获敌人四辆坦克']
for s in sents:
    print_sdp(s)

_(): 。
└── Root(Root_根节点): 吃
    ├── Agt(Agent_施事关系): 我
    ├── mDepd(_): 了
    └── Pat(Patient_受事关系): 冰淇淋
        ├── Meas(_): 个
        └── Mod(_): 味
            └── Mod(_): 巧克力
_(): 。
└── Root(Root_根节点): 去
    ├── Time(Time_时间角色): 今天
    ├── Agt(Agent_施事关系): 我
    ├── Loc(Location_空间角色): 电影院
    └── eEfft(_): 看
        ├── mDepd(_): 了
        └── Cont(Content_客事关系): 巧克力
            └── Mod(_): 浓情
_(): .
└── Root(Root_根节点): 熬粥
    ├── Agt(Agent_施事关系): 她
    └── Matl(Material_材料角色): 小米
        └── mRela(_): 用
_(): .
└── Root(Root_根节点): 熬粥
    ├── Agt(Agent_施事关系): 她
    └── Tool(Tool_工具角色): 砂锅
        ├── mRela(_): 用
        └── Mod(_): 美
            └── mDepd(_): 的
_(): .
└── Root(Root_根节点): 缴获
    ├── Agt(Agent_施事关系): 我军
    └── Pat(Patient_受事关系): 坦克
        ├── Exp(Experiencer_当事关系): 敌人
        └── Meas(_): 辆
            └── Meas(_): 四


In [21]:
sents=['一九九五年冬天', '一九九五年的冬天很冷', 
       '一九九五年是他倒霉的一年', '他说他的生日是九月二日']
for s in sents:
    print_sdp(s)

_(): .
└── Root(Root_根节点): 冬天
    └── Mod(_): 一九九五年
_(): .
└── Root(Root_根节点): 冷
    ├── Exp(Experiencer_当事关系): 冬天
    │   └── Time(Time_时间角色): 一九九五年
    │       └── mDepd(_): 的
    └── mDepd(_): 很
_(): .
└── Root(Root_根节点): 是
    ├── Exp(Experiencer_当事关系): 一九九五年
    └── Link(_): 年
        ├── Mod(_): 倒霉
        │   ├── Exp(Experiencer_当事关系): 他
        │   └── mDepd(_): 的
        └── Meas(_): 一
_(): .
└── Root(Root_根节点): 说
    ├── Agt(Agent_施事关系): 他
    └── dCont(_): 是
        ├── Exp(Experiencer_当事关系): 生日
        │   └── Exp(Experiencer_当事关系): 他
        │       └── mDepd(_): 的
        └── Link(_): 二日
            └── Mod(_): 九月


In [22]:
from ltp import LTP

ltp = LTP()

seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
sdp = ltp.sdp(hidden, graph=False)
sdp

[[(1, 2, 'Agt'),
  (2, 0, 'Root'),
  (3, 2, 'Datv'),
  (4, 2, 'eEfft'),
  (5, 4, 'eEfft'),
  (6, 5, 'Pat'),
  (7, 2, 'mPunc')]]

In [23]:
seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
sdp = ltp.sdp(hidden, graph=True)
sdp

[[(1, 2, 'Agt'),
  (2, 0, 'Root'),
  (3, 2, 'Datv'),
  (3, 4, 'Agt'),
  (3, 5, 'Agt'),
  (4, 2, 'eEfft'),
  (5, 4, 'eEfft'),
  (6, 5, 'Pat'),
  (7, 2, 'mPunc')]]

In [24]:
# seg, hidden = ltp.seg(["我军缴获敌人四辆坦克"])
# seg, hidden = ltp.seg(["这房子朝南"])
sents=['打篮球的小姑娘', '他听到鞭炮声', '他写了本小说', '周杰伦的出生日期是哪天']
for s in sents:
    seg, hidden = ltp.seg([s])
    sdp = ltp.sdp(hidden, graph=False)
    print(s, sdp)

打篮球的小姑娘 [[(1, 4, 'rAgt'), (2, 1, 'Pat'), (3, 1, 'mDepd'), (4, 0, 'Root')]]
他听到鞭炮声 [[(1, 2, 'Agt'), (2, 0, 'Root'), (3, 2, 'Cont')]]
他写了本小说 [[(1, 2, 'Agt'), (2, 0, 'Root'), (3, 2, 'mDepd'), (4, 5, 'Meas'), (5, 2, 'Prod')]]
周杰伦的出生日期是哪天 [[(1, 4, 'Exp'), (2, 1, 'mDepd'), (3, 4, 'rTime'), (4, 5, 'Exp'), (5, 0, 'Root'), (6, 7, 'Sco'), (7, 5, 'Link')]]


In [4]:
seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])
pos = ltp.pos(hidden)
seg, pos, ltp.ner(hidden)

([['他', '叫', '汤姆', '去', '拿', '外衣', '。']],
 [['r', 'v', 'nh', 'v', 'v', 'n', 'wp']],
 [[('Nh', 2, 2)]])

In [9]:
print(seg)
ltp.srl(hidden)[0]

[['他', '叫', '汤姆', '去', '拿', '外衣', '。']]


[[],
 [('A0', 0, 0), ('A1', 2, 2), ('A2', 3, 5)],
 [],
 [],
 [('A0', 2, 2), ('A1', 5, 5)],
 [],
 []]

In [5]:
ners=ltp.ner(hidden)
for tag, start, end in ners[0]:
    print(tag,":", "".join(seg[0][start:end + 1]))

Nh : 汤姆


In [21]:
from sagas.listings.co_data import SrlReform, CoReforms, SrlVerb
def norm_arg(tag, lead='B'):
    if len(tag)==2 and tag[0]=='A':
        return f"{lead}-ARG{tag[1]}"
    return f"{lead}-{tag}"
def print_args(sents):
    segs, hidden = ltp.seg([sents])
    srls=ltp.srl(hidden)
    seg=segs[0]
    srl=srls[0]
    
    reform=SrlReform(words=seg, verbs=[])
    for i, (tok, term) in enumerate(zip(seg, srl)):
        print(tok, term)
        if not term:
            continue
        
        placers=['O']*len(seg)
        for tag, start, end in term:
            for p in range(start,end+1): placers[p]=norm_arg(tag, 'I')
            placers[start]=norm_arg(tag)
            print('\t', tag, ":", "".join(seg[start:end + 1]))
        placers[i]='B-V'
        print('\t', placers)        
        verb_form=SrlVerb(verb=tok, description='', tags=placers)
        reform.verbs.append(verb_form)
    return reform

print_args("叫汤姆在中午去拿外衣。")

叫 [('A1', 1, 1), ('A2', 2, 6)]
	 A1 : 汤姆
	 A2 : 在中午去拿外衣
	 ['B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']
汤姆 []
在 []
中午 []
去 []
拿 [('A0', 1, 1), ('ARGM-TMP', 2, 3), ('A1', 6, 6)]
	 A0 : 汤姆
	 ARGM-TMP : 在中午
	 A1 : 外衣
	 ['O', 'B-ARG0', 'B-ARGM-TMP', 'I-ARGM-TMP', 'O', 'B-V', 'B-ARG1', 'O']
外衣 []
。 []


<SrlReform: verbs=[<SrlVerb: verb='叫', description='', tags=['B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']>, <SrlVerb: verb='拿', description='', tags=['O', 'B-ARG0', 'B-ARGM-TMP', 'I-ARGM-TMP', 'O', 'B-V', 'B-ARG1', 'O']>], words=['叫', '汤姆', '在', '中午', '去', '拿', '外衣', '。']>

In [28]:
print_args("叫汤姆不要拿外衣。")

叫 [('A1', 1, 1), ('A2', 2, 4)]
	 A1 : 汤姆
	 A2 : 不要拿外衣
汤姆 []
不要 []
拿 [('A0', 1, 1), ('ARGM-ADV', 2, 2), ('A1', 4, 4)]
	 A0 : 汤姆
	 ARGM-ADV : 不要
	 A1 : 外衣
外衣 []
。 []


In [29]:
seg[0][1]

'叫'

In [30]:
def proc_sents(sents):
    seg, hidden = ltp.seg([sents])
    sdp = ltp.sdp(hidden, graph=False)
    for start, end, tag in sdp[0]:
        print(seg[0][start-1],f"({tag})", seg[0][end-1])

proc_sents("他叫汤姆去拿外衣。")

他 (Agt) 叫
叫 (Root) 。
汤姆 (Datv) 叫
去 (eEfft) 叫
拿 (eEfft) 去
外衣 (Pat) 拿
。 (mPunc) 叫


In [31]:
proc_sents("他打了小明。")

他 (Agt) 打
打 (Root) 。
了 (mDepd) 打
小明 (Pat) 打
。 (mPunc) 打


In [32]:
proc_sents("小明被他打了。")
print('*'*15)
proc_sents("他打了小明一拳。")

小明 (Pat) 打
被 (mRela) 他
他 (Agt) 打
打 (Root) 。
了 (mDepd) 打
。 (mPunc) 打
***************
他 (Agt) 打
打 (Root) 。
了 (mDepd) 打
小明 (Pat) 打
一 (Meas) 拳
拳 (Meas) 打
。 (mPunc) 打


In [33]:
# ltp.srl(hidden, keep_empty=False)
ltp.srl(hidden)

[[[],
  [('A0', 0, 0), ('A1', 2, 2), ('A2', 3, 5)],
  [],
  [],
  [('A0', 2, 2), ('A1', 5, 5)],
  [],
  []]]

In [34]:
print_sdp("我爱自然语言处理。元芳你怎么看")

_(): .
└── Root(Root_根节点): 爱
    ├── Agt(Agent_施事关系): 我
    ├── Cont(Content_客事关系): 处理
    │   └── Tool(Tool_工具角色): 语言
    │       └── Mod(_): 自然
    └── eAdvt(event adversative_转折关系): 看
        ├── Agt(Agent_施事关系): 元芳
        ├── Agt(Agent_施事关系): 你
        └── Mann(Manner_方式角色): 怎么


In [35]:
print_sdp("2018年4月22日，2017年全球最赚钱企业排行榜第15。")
print_sdp("微软是2018年4月22日，2017年全球最赚钱企业排行榜第15。")
print_sdp("2018年4月22日，微软是2017年全球最赚钱企业排行榜第15。")
print_args("微软是2018年4月22日，2017年全球最赚钱企业排行榜第15。")

_(): 。
└── Root(Root_根节点): 第 15
    ├── Time(Time_时间角色): 22日
    │   ├── Mod(_): 2018年
    │   └── Mod(_): 4月
    └── Exp(Experiencer_当事关系): 排行榜
        ├── Time(Time_时间角色): 2017年
        └── Mod(_): 企业
            ├── Sco(Scope_范围角色): 全球
            └── Mod(_): 赚钱
                └── mDepd(_): 最
_(): 。
└── Root(Root_根节点): 是
    ├── Exp(Experiencer_当事关系): 微软
    ├── Link(_): 22日
    │   ├── Mod(_): 2018年
    │   └── Mod(_): 4月
    └── Link(_): 第 15
        └── Exp(Experiencer_当事关系): 排行榜
            └── Mod(_): 企业
                ├── Time(Time_时间角色): 2017年
                ├── Sco(Scope_范围角色): 全球
                └── Mod(_): 赚钱
                    └── mDepd(_): 最
_(): 。
└── Root(Root_根节点): 是
    ├── Time(Time_时间角色): 22日
    │   ├── Mod(_): 2018年
    │   └── Mod(_): 4月
    ├── Exp(Experiencer_当事关系): 微软
    └── Link(_): 第 15
        └── Exp(Experiencer_当事关系): 排行榜
            ├── Time(Time_时间角色): 2017年
            ├── Sco(Scope_范围角色): 全球
            └── Mod(_): 企业
                └── Mod(_):

In [36]:
print_sdp("2018年4月22日全球最赚钱企业排行榜第15。")

_(): 。
└── Root(Root_根节点): 第 15
    ├── Time(Time_时间角色): 22日
    │   ├── Mod(_): 2018年
    │   └── Mod(_): 4月
    └── Sco(Scope_范围角色): 排行榜
        ├── Sco(Scope_范围角色): 全球
        └── Mod(_): 企业
            └── Mod(_): 赚钱
                └── mDepd(_): 最


In [37]:
print_args("2018年4月22日，2017年全球最赚钱企业排行榜第15。")

2018年 []
4月 []
22日 []
， []
2017年 []
全球 []
最 []
赚钱 [('ARGM-LOC', 5, 5), ('ARGM-ADV', 6, 6), ('A0', 8, 8)]
	 ARGM-LOC : 全球
	 ARGM-ADV : 最
	 A0 : 企业
企业 []
排行榜 []
第 15 []
。 []


In [38]:
print_args("2018年4月22日，微软是2017年全球最赚钱企业排行榜第15。")

2018年 []
4月 []
22日 []
， []
微软 []
是 [('ARGM-TMP', 0, 2), ('A0', 4, 4), ('A1', 6, 12)]
	 ARGM-TMP : 2018年4月22日
	 A0 : 微软
	 A1 : 2017年全球最赚钱企业排行榜第 15
2017年 []
全球 []
最 []
赚钱 [('ARGM-TMP', 6, 6), ('ARGM-LOC', 7, 7), ('ARGM-ADV', 8, 8), ('A0', 10, 10)]
	 ARGM-TMP : 2017年
	 ARGM-LOC : 全球
	 ARGM-ADV : 最
	 A0 : 企业
企业 []
排行榜 []
第 15 []
。 []


In [39]:
print_args("2019年6月，微软悄然删除其MS Celeb人脸识别数据库，微软称该数据库是全球最大的公开人脸识别数据库。")

2019年 []
6月 []
， []
微软 []
悄然 []
删除 [('ARGM-TMP', 0, 1), ('A0', 3, 3), ('ARGM-ADV', 4, 4), ('A1', 6, 11)]
	 ARGM-TMP : 2019年6月
	 A0 : 微软
	 ARGM-ADV : 悄然
	 A1 : 其MS Celeb人脸识别数据库
其 []
MS Celeb []
人 []
脸 []
识别 []
数据库 []
， []
微软 []
称 [('A0', 13, 13), ('A1', 15, 25)]
	 A0 : 微软
	 A1 : 该数据库是全球最大的公开人脸识别数据库
该 []
数据库 []
是 [('A0', 15, 16), ('A1', 18, 25)]
	 A0 : 该数据库
	 A1 : 全球最大的公开人脸识别数据库
全球 []
最 []
大 [('ARGM-LOC', 18, 18), ('ARGM-ADV', 19, 19), ('A0', 22, 25)]
	 ARGM-LOC : 全球
	 ARGM-ADV : 最
	 A0 : 公开人脸识别数据库
的 []
公开人 []
脸 []
识别 []
数据库 []
。 []


In [40]:
print_sdp("2018年4月22日，2017年全球最赚钱企业排行榜第15。2018年5月29日，《2018年BrandZ全球最具价值品牌100强》第4位。2018年7月19日，《财富》世界500强排行榜位列71位。2018年12月18日，《2018世界品牌500强》第4位。")

_(): 。
└── Root(Root_根节点): 位
    ├── Meas(_): 第 15
    │   └── Exp(Experiencer_当事关系): 排行榜
    │       ├── Time(Time_时间角色): 22日
    │       │   ├── Mod(_): 2018年
    │       │   └── Mod(_): 4月
    │       ├── Time(Time_时间角色): 2017年
    │       ├── Sco(Scope_范围角色): 全球
    │       └── Exp(Experiencer_当事关系): 企业
    │           └── Mod(_): 赚钱
    │               └── mDepd(_): 最
    ├── Time(Time_时间角色): 29日
    │   ├── Mod(_): 2018年
    │   └── Mod(_): 5月
    ├── dMod(_): 具
    │   ├── Time(Time_时间角色): 2018年
    │   ├── Loc(Location_空间角色): 全球
    │   │   └── Mod(_): BrandZ
    │   ├── mDepd(_): 最
    │   ├── Link(_): 品牌
    │   │   └── Mod(_): 价值
    │   └── Link(_): 强
    │       └── Meas(_): 100
    ├── Meas(_): 第 4
    └── eCtnu(_): 列
        ├── Time(Time_时间角色): 19日
        │   ├── Mod(_): 2018年
        │   └── Mod(_): 7月
        ├── Exp(Experiencer_当事关系): 排行榜
        │   ├── Mod(_): 财富
        │   ├── Mod(_): 世界
        │   └── Mod(_): 强
        │       └── Meas(_): 500
        ├── Exp(

In [41]:
print_sdp('张三在美国用筷子吃川菜。')

_(): 。
└── Root(Root_根节点): 吃
    ├── Agt(Agent_施事关系): 张三
    ├── Loc(Location_空间角色): 美国
    │   ├── mRela(_): 在
    │   └── mRela(_): 用
    │       └── Pat(Patient_受事关系): 筷子
    └── Pat(Patient_受事关系): 川菜


In [43]:
print_sdp('将该平台用于商业目的（如企业合作项目等）则需要付费。')
print_args('将该平台用于商业目的（如企业合作项目等）则需要付费。')

_(): 。
└── Root(Root_根节点): 付费
    ├── dTime(_): 用于
    │   ├── Pat(Patient_受事关系): 平台
    │   │   ├── mRela(_): 将
    │   │   └── Sco(Scope_范围角色): 该
    │   └── Datv(Dative_涉事关系): 目的
    │       ├── Mod(_): 商业
    │       └── Mod(_): 项目
    │           ├── mDepd(_): 如
    │           ├── dMod(_): 合作
    │           │   └── Agt(Agent_施事关系): 企业
    │           └── mDepd(_): 等
    ├── mDepd(_): 则
    └── mDepd(_): 需要
将 []
该 []
平台 []
用于 [('A1', 1, 2), ('A2', 4, 11)]
	 A1 : 该平台
	 A2 : 商业目的（如企业合作项目等
商业 []
目的 []
（ []
如 []
企业 []
合作 []
项目 []
等 []
） []
则 []
需要 []
付费 [('A1', 0, 12), ('ARGM-ADV', 13, 13)]
	 A1 : 将该平台用于商业目的（如企业合作项目等）
	 ARGM-ADV : 则
。 []
